###Loading libraries

In [ ]:
#install missing modules of google colab (geopandas)
!pip install geopandas

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 15.8MB/s 
     |████████████████████████████████| 14.8MB 308kB/s 


In [ ]:
#import libraries
import branca.colormap as cm
import folium
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
#link google drive files to google colab
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Data preparation


In [ ]:
#load csv file on covid_19
corona_df = pd.read_csv("/content/gdrive/My Drive/colab_notebooks/dataset/covid_19_data.csv")

In [ ]:
#load shp of world countries
countries = gpd.read_file("/content/gdrive/My Drive/colab_notebooks/dataset/world_countries/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp")

In [ ]:
#match names of csv to shp
corona_df = corona_df.replace({'Country/Region' : 
                      dict.fromkeys(['Taiwan', 
                                     'Mainland China', 
                                     'Hong Kong', 
                                     'Macau'], 
                                     'China')})
corona_df = corona_df.replace({'Country/Region' : 'US'}, 
                                'United States')
corona_df = corona_df.replace({'Country/Region' : 'UK'}, 
                                'United Kingdom')
corona_df = corona_df.replace({'Country/Region' : 'North Ireland'}, 
                                'United Kingdom')
corona_df = corona_df.replace({'Country/Region' : 'Republic of Ireland'}, 
                                'Ireland')
corona_df = corona_df.replace({'Country/Region' : 'Vatican City'}, 
                                'Italy')
countries = countries.replace({'CNTRY_NAME' : 'Byelarus'}, 
                               'Belarus')
countries = countries.replace({'CNTRY_NAME' : 'Macedonia'}, 
                               'North Macedonia')

In [ ]:
#match country names
countries = countries.rename(columns={'CNTRY_NAME': 'Country/Region'})

In [ ]:
#remove countries with 0 cases of covid_19
corona_df = corona_df[corona_df.Confirmed != 0]

In [ ]:
#sort data by country and rest index
sorted_df = corona_df.sort_values(['Country/Region', 
                     'ObservationDate']).reset_index(drop=True)

In [ ]:
#group number of cases by country
sum_df = sorted_df.groupby(['Country/Region', 'ObservationDate'], as_index=False).sum()

In [ ]:
#join csv and shp files
joined_df = sum_df.merge(countries, on='Country/Region')

In [ ]:
#use log of the number of confirmed cases for each country
joined_df['log_Confirmed'] = np.log10(joined_df['Confirmed'])

In [ ]:
#convert the ObservationDate from unix time in nanoseconds
joined_df['date_sec'] = pd.to_datetime(joined_df['ObservationDate']).astype(int) / 10**9
joined_df['date_sec'] = joined_df['date_sec'].astype(int).astype(str)

In [ ]:
#select the columns needed for the map
joined_df = joined_df[['Country/Region', 'date_sec', 'log_Confirmed', 'geometry']]

###Creation of map

In [ ]:
#set up map folium
mymap = folium.Map(tiles='cartodbpositron')
mymap.save(outfile='infinite_scroll.html')

In [ ]:
#add up country boundaries
mymap_fix_boundary = folium.Map(min_zoom=2, max_bounds=True, tiles='cartodbpositron')
mymap_fix_boundary.save(outfile='fix_boundary.html')

In [ ]:
#define color map
max_colour = max(joined_df['log_Confirmed'])
min_colour = min(joined_df['log_Confirmed'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour'] = joined_df['log_Confirmed'].map(cmap)

In [ ]:
#construct dictionary
country_list = joined_df['Country/Region'].unique().tolist()
country_idx = range(len(country_list))

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = joined_df[joined_df['Country/Region'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

In [ ]:
#make dataframe containing the features for each country
countries_df = joined_df[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().reset_index()

In [ ]:
#create map and add a colorbar
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')

_ = TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Log of number of confirmed cases"
slider_map.save(outfile="/content/gdrive/My Drive/colab_notebooks/TimeSliderChoropleth.html")

Reference link:
https://www.jumpingrivers.com/blog/interactive-maps-python-covid-19-spread/

Data link:
https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset#covid_19_data.csv